# Assignment 3

## Instructions
- Run this notebook on ```Google Colab(preferable)```
- Write your code and analysis in the indicated cells.
- Ensure that this notebook runs without errors when the cells are run in sequence.
- Do not attempt to change the contents of other cells. 

## Packages Used
- sklearn [link](https://scikit-learn.org/)
- Keras [link](https://keras.io/guides/)

## Submission
- Rename the notebook to `<roll_number>_Assignment3_Q3.ipynb`.


## Question 3
Fake news is a widespread problem and there are many methods for combating it.
You have to build a fake news detection system using a ML model. Train any ML model (ANN, LSTM) over the given Dataset.
The dataset has short statements spoken by people and has the meta-information and corresponding label for those sentences. 
Your target is label column which has 6 labels(in the increasing order of truthfullness): pants-fire, false, barely-true, half-true, mostly-true, true.

The features are 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c', 'pants_on_fire_c', 'venue' and the target is column "label".

The statement is made by speaker whose job, party are given along with 6 columns which are an account of the  type of news(labels) the person has shared before. 
The person who has shared fake content before is likely to share it in future and this can be accounted by the ML model as a feature. Column barely_true_c contains how many barely_true news has the speaker shared (and so is with column X_c, value of X_c is number of X the person shared).


You have to perform two tasks:
* task1: Binary classification <br>
Classify the given news as true/false. Take the labels pants-fire, false, barely-true as false and rest (half-true, mostly-true, true) as true.
* task2: Six-way classification <br>
Classify the given news into six-classes "pants-fire, false, barely-true, half-true, mostly-true, true".

For each of the tasks:
1) Experiment with depth of network and try to fine-tune hyperparameters reporting your observations. <br>
2) Report the accuracy, f1-score, confusion matrix on train, val and test sets. <br>
3) Experiment with bag-of-words, glove and bert embeddings(code given in the below notebook) and report results. <br> Comment on what is the affect of embedding on the results.

The pre-processing code is provided, you need to write the training and test.

Note: You are supposed to train on trainset, fine-tune on val and just eval on test set. If found that you trained on val/test sets, the penalty will be incurred.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/smai/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/smai


In [1]:
!pip install numpy
!pip install tensorflow
!pip install re
!pip install nltk
!pip install keras
!pip install sklearn

     |████████████████████████████████| 462 kB 5.0 MB/s 
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [2]:
!pip install sentence_transformers

     |████████████████████████████████| 79 kB 3.8 MB/s 
     |████████████████████████████████| 4.0 MB 32.6 MB/s 
     |████████████████████████████████| 1.2 MB 40.8 MB/s 
     |████████████████████████████████| 77 kB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 28.0 MB/s 
     |████████████████████████████████| 895 kB 42.1 MB/s 
     |████████████████████████████████| 596 kB 36.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=0ca9552b507db0c3a2ca58a57cedfb76b8617c5400d7eb1cbca72340c5713278
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
# Importing libraries
import numpy as np
import pandas as pd
from tensorflow import keras  #feel free to use any other library
import numpy as np

import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from keras.utils import np_utils


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
from zipfile import ZipFile
file_name = "/content/q3_data.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()

In [5]:
train = pd.read_csv('q3_data/train.csv')
val = pd.read_csv('q3_data/val.csv')
test = pd.read_csv('q3_data/test.csv')

In [6]:
# Dropping the 'id' column
train.drop('id', axis = 1, inplace = True)
test.drop('id', axis = 1, inplace = True)
val.drop('id', axis = 1, inplace = True)

In [7]:
train.head(5)

,label,statement,subject,speaker,job,state,party,barely_true_c,false_c,half_true_c,mostly_true_c,pants_on_fire_c,venue
0,False,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0,0,0,a mailer
1,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0,0,1,1,0,a floor speech.
2,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70,71,160,163,9,Denver
3,False,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7,19,3,5,44,a news release
4,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15,9,20,19,2,an interview on CNN


In [8]:
# Checking the shape of data
print(train.shape)
print(val.shape)
print(test.shape)

(10269, 13)
(1284, 13)
(1283, 13)


## Clean and pre-process data
* Replace missing values
* Remove numbers and special characters
* Convert to upper-case

We experiment with two types of processing, one directly appending the other attributes like subject, job, state, party to sentence and then applying bag of words on it.

Other being encoding sentence with glove embeddings and passing just that.

In [9]:

def dataPreprocessing(data):
    '''Function for cleaning the dataset
    '''
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = statement.split()
        
        # you can experiment with any other stemmers
        ps = PorterStemmer()
        statement = [ps.stem(word) for word in statement if not word in set(stopwords.words('english'))] # Stemming the dataset and removing stopwords
        statement = ' '.join(statement)
        subject = data['subject'][x].replace(',', ' ')
        speaker = data['speaker'][x]
        job = data['job'][x].lower()
        # job = job.replace(' ', '-')
        state = data['state'][x].lower()
        party = data['party'][x].lower()
        corpus.append(statement + ' '  + subject + ' ' + job + ' ' + state + ' ' + party)
    return corpus

In [10]:
x_train = dataPreprocessing(train)
x_val = dataPreprocessing(val) 
x_test = dataPreprocessing(test) 

In [11]:
len(x_train), len(x_val), len(x_test)

(10269, 1284, 1283)

In [12]:
corpus = x_train + x_val + x_test

## Using bag-of-words embedding


In [13]:
# Converting the corpus into bag-of-words
cv = CountVectorizer(max_features = 8000)
X = cv.fit_transform(corpus).toarray()

In [14]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [15]:
X.shape

(12836, 8000)

In [16]:
train.columns

Index(['label', 'statement', 'subject', 'speaker', 'job', 'state', 'party',
       'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c', 'venue'],
      dtype='object')

In [17]:
# Selecting the columns 'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c'
label_cols = ['barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c']
x_train2 = train[label_cols]
x_val2 = val[label_cols]
x_test2 = test[label_cols]

In [18]:
# Stacking x_train and x_train2 horizontally
x_train_bow = np.hstack((X[:len(x_train)], x_train2))
x_val_bow = np.hstack((X[len(x_train):len(x_train)+len(x_val)], x_val2))
x_test_bow = np.hstack((X[len(x_train)+len(x_val):], x_test2))

In [19]:
x_train_bow.shape

(10269, 8005)

## Use of Glove Embedding


In [21]:
!wget 'https://nlp.stanford.edu/data/glove.6B.zip'


--2022-04-14 15:17:21--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-04-14 15:17:21--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.15MB/s    in 2m 40s  

2022-04-14 15:20:01 (5.15 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



download glove embeddings from 'https://nlp.stanford.edu/data/glove.6B.zip','glove.6B.zip'
and place in your current working folder


In [22]:
!unzip "glove.6B.zip" -d "glove"

Archive:  glove.6B.zip
  inflating: glove/glove.6B.50d.txt  
  inflating: glove/glove.6B.100d.txt  
  inflating: glove/glove.6B.200d.txt  
  inflating: glove/glove.6B.300d.txt  


In [23]:
emmbed_dict = {}
with open('glove/glove.6B.200d.txt','r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbed_dict[word]=vector


In [24]:
emmbed_dict['oov'] = np.zeros(200)

In [25]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [26]:
import tensorflow as tf

In [28]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
def dataPreprocessing_glove(data):
    corpus = []
    # Missing values
    data["job"].fillna("no-job", inplace = True)
    data["state"].fillna("no-state", inplace = True)

    for x in range(data.shape[0]):
        statement = re.sub('[^a-zA-Z]', ' ', data['statement'][x]) # Removing all numbers and special characters
        statement = statement.lower() # Converting uppercase to lowercase
        statement = word_tokenize(statement)

        embed_statement = []
        for w in statement:
            if w in emmbed_dict:
                embed_statement.append(emmbed_dict[w])
            else:
                embed_statement.append(emmbed_dict['oov'])
         
        # bonus: Think how you can encode the below features(hint: look upon label encoding or training your own word2vec or any other embedding model)
    
#         subject = data['subject'][x].replace(',', ' ')
#         speaker = data['speaker'][x]
#         job = data['job'][x].lower()
#         # job = job.replace(' ', '-')
#         state = data['state'][x].lower()
#         party = data['party'][x].lower()
        corpus.append(embed_statement)
    total_size=50
    corpus=np.array(corpus)
    corpus= tf.keras.preprocessing.sequence.pad_sequences(
    corpus,
    maxlen=total_size,
    padding='pre'
)
    return corpus

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
x_train_glove = dataPreprocessing_glove(train)
x_val_glove = dataPreprocessing_glove(val) 
x_test_glove = dataPreprocessing_glove(test) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [30]:
# Printing the shape of the data
print(x_train_glove.shape)

(10269, 50, 200)


In [31]:
# Reshaping the data to resolve errors and caching issue in ram
x_train_glove=x_train_glove.reshape(x_train_glove.shape[0],50*x_train_glove.shape[2])
x_val_glove=x_val_glove.reshape(x_val_glove.shape[0],50*x_val_glove.shape[2])
x_test_glove=x_test_glove.reshape(x_test_glove.shape[0],50*x_test_glove.shape[2])

In [32]:
print(x_train_glove.shape)

(10269, 10000)


## Use of bert embeddings
note: we used our pre-processed code for bow which has the attributed appended to end the end of sentence. 

In [33]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

x_train_bert = np.hstack((model.encode(x_train), x_train2))
x_val_bert = np.hstack((model.encode(x_val), x_val2))
x_test_bert = np.hstack((model.encode(x_test), x_test2))

Now use the above 3 types of embedded inputs(bow, glove, bert embeddings) for the 2 classification tasks and compare their outputs


# Six-way classification

## Preprocessing

In [34]:
num_classes = 6
# Preprocessing function for the labels
def categorize(data):
    y = data["label"].tolist()

    # Encoding the Dependent Variable
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)

    # Converting to binary class matrix
    y = np_utils.to_categorical(y, num_classes)
    return y

In [35]:
y_train_six_way = categorize(train)
y_test_six_way = categorize(test)
y_val_six_way = categorize(val)
print(len(y_train_six_way))

10269


Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


## Model

In [37]:
from keras.models import Sequential
from sklearn.metrics import classification_report,accuracy_score, precision_score, recall_score, f1_score,confusion_matrix,multilabel_confusion_matrix
from keras.layers import LSTM,Dense, Dropout, Embedding, Bidirectional
from keras.preprocessing import sequence



In [39]:
## write your code here
# Initialize hyperparameters
# Create model
model_bert = Sequential()
model_bert.add(LSTM(128, return_sequences = True, input_shape = (x_train_bert.shape[1], 1)))
model_bert.add(Dropout(0.5))
model_bert.add(LSTM(64))
model_bert.add(Dropout(0.4))
model_bert.add(Dense(32))
model_bert.add(Dropout(0.3))
model_bert.add(Dense(6, activation='softmax'))
model_bert.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
# train
history =model_bert.fit(x_train_bert, y_train_six_way, validation_data=(x_val_bert, y_val_six_way), batch_size = 128, epochs = 12)
# test


# report accuracy, f1-score and confusion matrix

Epoch 1/12
81/81 [==============================] - 21s 159ms/step - loss: 1.7710 - accuracy: 0.2012 - val_loss: 1.7478 - val_accuracy: 0.1893
Epoch 2/12
81/81 [==============================] - 12s 147ms/step - loss: 1.7459 - accuracy: 0.2184 - val_loss: 1.7179 - val_accuracy: 0.2477
Epoch 3/12
81/81 [==============================] - 12s 144ms/step - loss: 1.7208 - accuracy: 0.2410 - val_loss: 1.6797 - val_accuracy: 0.2702
Epoch 4/12
81/81 [==============================] - 12s 144ms/step - loss: 1.6870 - accuracy: 0.2607 - val_loss: 1.6342 - val_accuracy: 0.3162
Epoch 5/12
81/81 [==============================] - 12s 144ms/step - loss: 1.6347 - accuracy: 0.2988 - val_loss: 1.5661 - val_accuracy: 0.3676
Epoch 6/12
81/81 [==============================] - 12s 144ms/step - loss: 1.5405 - accuracy: 0.3550 - val_loss: 1.4193 - val_accuracy: 0.4019
Epoch 7/12
81/81 [==============================] - 12s 145ms/step - loss: 1.4330 - accuracy: 0.4070 - val_loss: 1.3524 - val_accuracy: 0.4260

ValueError: ignored

In [40]:
print("Result on test data")
print("loss on test data, test data accuracy :", model_bert.evaluate(x_test_bert, y_test_six_way))

Result on test data
41/41 [==============================] - 3s 61ms/step - loss: 1.3217 - accuracy: 0.4474
loss on test data, test data accuracy : [1.3217107057571411, 0.4473889470100403]


In [41]:
y_hats =model_bert.predict(x_test_bert)
actual_y=np.argmax(y_test_six_way,axis=1)
pred_y=np.argmax(y_hats,axis=1)
con_matrix = confusion_matrix(actual_y,pred_y)
print("Confusion Matrix\n",con_matrix)
print('\nAccuracy on Bert embedded inputs : {:.4f}\n'.format(accuracy_score(actual_y,pred_y)))
print('\n f1 score on Bert embedded inputs : {:.4f}\n'.format(f1_score(actual_y, pred_y,average='macro')))
print('\nClassification Report of Bert embedded inputs \n')
print(classification_report(actual_y, pred_y, target_names=['False', 'True', 'barely-true', 'half-true', 'mostly-true', 'pants-fire']))

Confusion Matrix
 [[145   2   8  61  25   9]
 [ 41  33  13  81  40   3]
 [ 67   0  57  71  13   6]
 [ 47   1   3 181  35   0]
 [ 38   0   6  89 114   2]
 [ 33   0   1  11   3  44]]

Accuracy on Bert embedded inputs : 0.4474


 f1 score on Bert embedded inputs : 0.4379


Classification Report of Bert embedded inputs 

              precision    recall  f1-score   support

       False       0.39      0.58      0.47       250
        True       0.92      0.16      0.27       211
 barely-true       0.65      0.27      0.38       214
   half-true       0.37      0.68      0.48       267
 mostly-true       0.50      0.46      0.48       249
  pants-fire       0.69      0.48      0.56        92

    accuracy                           0.45      1283
   macro avg       0.58      0.44      0.44      1283
weighted avg       0.56      0.45      0.43      1283



In [43]:
## write your code here
# Initialize hyperparameters
# Create model
model_bow = Sequential()
model_bow.add(LSTM(256, return_sequences = True, input_shape = (x_train_bow.shape[1], 1)))
model_bow.add(Dropout(0.5))
model_bow.add(LSTM(128))
model_bow.add(Dropout(0.4))
model_bow.add(Dense(64))
model_bow.add(Dropout(0.3))
model_bow.add(Dense(6, activation='softmax'))
model_bow.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
# train
history =model_bow.fit(x_train_bow, y_train_six_way, validation_data=(x_val_bow, y_val_six_way), batch_size = 64, epochs = 12)
# test


# report accuracy, f1-score and confusion matrix

Epoch 1/12
161/161 [==============================] - 500s 3s/step - loss: 1.7585 - accuracy: 0.2099 - val_loss: 1.6991 - val_accuracy: 0.2438
Epoch 2/12
161/161 [==============================] - 494s 3s/step - loss: 1.7339 - accuracy: 0.2274 - val_loss: 1.6916 - val_accuracy: 0.2438
Epoch 3/12
161/161 [==============================] - 493s 3s/step - loss: 1.6567 - accuracy: 0.2970 - val_loss: 1.5725 - val_accuracy: 0.3832
Epoch 4/12
161/161 [==============================] - 493s 3s/step - loss: 1.4880 - accuracy: 0.3892 - val_loss: 1.3653 - val_accuracy: 0.4330
Epoch 5/12
161/161 [==============================] - 493s 3s/step - loss: 1.3725 - accuracy: 0.4196 - val_loss: 1.3119 - val_accuracy: 0.4463
Epoch 6/12
161/161 [==============================] - 493s 3s/step - loss: 1.3511 - accuracy: 0.4315 - val_loss: 1.3261 - val_accuracy: 0.4315
Epoch 7/12
161/161 [==============================] - 494s 3s/step - loss: 1.3366 - accuracy: 0.4326 - val_loss: 1.2911 - val_accuracy: 0.4447

ValueError: ignored

In [44]:
print("Result on test data")
print("loss on test data, test data accuracy :", model_bow.evaluate(y_test_six_way, y_test_six_way))
y_hats =model_bow.predict(x_test_bow)
actual_y=np.argmax(y_test_six_way,axis=1)
pred_y=np.argmax(y_hats,axis=1)
con_matrix = confusion_matrix(actual_y,pred_y)
print("Confusion Matrix\n",con_matrix)
print('\nAccuracy on Bert embedded inputs : {:.4f}\n'.format(accuracy_score(actual_y,pred_y)))
print('\n f1 score on Bert embedded inputs : {:.4f}\n'.format(f1_score(actual_y, pred_y,average='macro')))
print('\nClassification Report of Bert embedded inputs \n')
print(classification_report(actual_y, pred_y, target_names=['False', 'True', 'barely-true', 'half-true', 'mostly-true', 'pants-fire']))

Confusion Matrix
 [[138   2  13  61  27   9]
 [ 38  33  17  46  74   3]
 [ 51   0  73  53  33   4]
 [ 45   1  13 146  62   0]
 [ 40   0  11  64 132   2]
 [ 30   0   3  11   4  44]]

Accuracy on Bert embedded inputs : 0.4412


 f1 score on Bert embedded inputs : 0.4390


Classification Report of Bert embedded inputs 

              precision    recall  f1-score   support

       False       0.40      0.55      0.47       250
        True       0.92      0.16      0.27       211
 barely-true       0.56      0.34      0.42       214
   half-true       0.38      0.55      0.45       267
 mostly-true       0.40      0.53      0.45       249
  pants-fire       0.71      0.48      0.57        92

    accuracy                           0.44      1283
   macro avg       0.56      0.43      0.44      1283
weighted avg       0.53      0.44      0.43      1283



In [1]:
## write your code here
# Initialize hyperparameters
# Create model
model_glove = Sequential()
model_glove.add(LSTM(80, return_sequences = True, input_shape = (x_train_glove.shape[1], 1)))
model_glove.add(Dropout(0.5))
model_glove.add(LSTM(60))
model_glove.add(Dropout(0.4))
model_glove.add(Dense(32))
model_glove.add(Dropout(0.3))
model_glove.add(Dense(6, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
# train
history =model_glove.fit(x_train_glove, y_train_six_way, validation_data=(x_val_glove, y_val_six_way), batch_size = 32, epochs = 12)
# test

y_hats =model_glove.predict(x_test_glove)
actual_y=np.argmax(y_test_six_way,axis=1)
pred_y=np.argmax(y_hats,axis=1)
con_matrix = confusion_matrix(actual_y,pred_y)
print("Confusion Matrix\n",con_matrix)
print('\nAccuracy on glove embedded inputs : {:.4f}\n'.format(accuracy_score(actual_y,pred_y)))
print('\n f1 score on glove embedded inputs : {:.4f}\n'.format(f1_score(actual_y, pred_y,average='macro')))
print('\nClassification Report of glove embedded inputs \n')
print(classification_report(actual_y, pred_y, target_names=['False', 'True', 'barely-true', 'half-true', 'mostly-true', 'pants-fire']))
# report accuracy, f1-score and confusion matrix

NameError: ignored

# Binary Classification

## Preprocessing

In [ ]:
num_classes = 2

In [ ]:
# Function for preprocessing labels
def dataPreprocessingBinary(data):
    y = data["label"].tolist()

    # Changing the 'half-true', 'mostly-true', barely-true', 'pants-fire' labels to True/False for Binary Classification
    for x in range(len(y)):
        if(y[x] == 'half-true'):
            y[x] = 'True'
        elif(y[x] == 'mostly-true'):
            y[x] = 'True'
        elif(y[x] == 'barely-true'):
            y[x] = 'False'
        elif(y[x] == 'pants-fire'):
            y[x] = 'False'

    # Converting the lables into binary class matrix
    labelencoder_y = LabelEncoder()
    y = labelencoder_y.fit_transform(y)
    y = np_utils.to_categorical(y, num_classes)
    return y

In [ ]:
y_train_binary = dataPreprocessingBinary(train)
y_test_binary = dataPreprocessingBinary(test)
y_val_binary = dataPreprocessingBinary(val)

## Model
Build a model and pass bow, glove and bert embedded inputs: x_train_bow, x_train_glove, x_train_bert(similarly validate for val and report results on test)


In [ ]:
## write your code here
# Initialize hyperparameters
# Create model
## write your code here
# Initialize hyperparameters
# Create model
model_bert_bin = Sequential()
model_bert_bin.add(LSTM(128, return_sequences = True, input_shape = (x_train_bert.shape[1], 1)))
model_bert_bin.add(Dropout(0.5))
model_bert_bin.add(LSTM(64))
model_bert_bin.add(Dropout(0.4))
model_bert_bin.add(Dense(32))
model_bert_bin.add(Dropout(0.3))
model_bert_bin.add(Dense(2, activation='softmax'))
model_bert_bin.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
# train
history =model_bert_bin.fit(x_train_bert, y_train_binary, validation_data=(x_val_bert, y_val_binary), batch_size = 128, epochs = 12)
# test
print("Result on test data")
print("loss on test data, test data accuracy :", model_bert_bin.evaluate(x_test_bert, y_test_binary))
y_hats =model_bert.predict(x_test_bert)
actual_y=np.argmax(y_test_binary,axis=1)
pred_y=np.argmax(y_hats,axis=1)
con_matrix = confusion_matrix(actual_y,pred_y)
print("Confusion Matrix\n",con_matrix)
print('\nAccuracy on Bert embedded inputs : {:.4f}\n'.format(accuracy_score(actual_y,pred_y)))
print('\n f1 score on Bert embedded inputs : {:.4f}\n'.format(f1_score(actual_y, pred_y,average='macro')))
print('\nClassification Report of Bert embedded inputs \n')
print(classification_report(actual_y, pred_y, target_names=['False', 'True', 'barely-true', 'half-true', 'mostly-true', 'pants-fire']))
# report accuracy, f1-score and confusion matrix
# train
# test
# report accuracy, f1-score and confusion matrix

In [ ]:
## write your code here
# Initialize hyperparameters
# Create model
## write your code here
# Initialize hyperparameters
# Create model
model_bow_bin = Sequential()
model_bow_bin.add(LSTM(256, return_sequences = True, input_shape = (x_train_bow.shape[1], 1)))
model_bow_bin.add(Dropout(0.5))
model_bow_bin.add(LSTM(128))
model_bow_bin.add(Dropout(0.4))
model_bow_bin.add(Dense(64))
model_bow_bin.add(Dropout(0.3))
model_bow_bin.add(Dense(2, activation='softmax'))
model_bow_bin.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
# train
history =model_bow_bin.fit(x_train_bow, y_train_binary, validation_data=(x_val_bow, y_val_binary), batch_size = 64, epochs = 12)
# test
print("Result on test data")
print("loss on test data, test data accuracy :", model_bow_bin.evaluate(x_test_bow, y_test_binary))
y_hats =model_bert.predict(x_test_bow)
actual_y=np.argmax(y_test_binary,axis=1)
pred_y=np.argmax(y_hats,axis=1)
con_matrix = confusion_matrix(actual_y,pred_y)
print("Confusion Matrix\n",con_matrix)
print('\nAccuracy on Bert embedded inputs : {:.4f}\n'.format(accuracy_score(actual_y,pred_y)))
print('\n f1 score on Bert embedded inputs : {:.4f}\n'.format(f1_score(actual_y, pred_y,average='macro')))
print('\nClassification Report of Bert embedded inputs \n')
print(classification_report(actual_y, pred_y, target_names=['False', 'True', 'barely-true', 'half-true', 'mostly-true', 'pants-fire']))
# report accuracy, f1-score and confusion matrix
# train
# test
# report accuracy, f1-score and confusion matrix

In [ ]:
## write your code here
# Initialize hyperparameters
# Create model
model_glove_bin = Sequential()
model_glove_bin.add(LSTM(80, return_sequences = True, input_shape = (x_train_glove.shape[1], 1)))
model_glove_bin.add(Dropout(0.5))
model_glove_bin.add(LSTM(60))
model_glove_bin.add(Dropout(0.4))
model_glove_bin.add(Dense(32))
model_glove_bin.add(Dropout(0.3))
model_glove_bin.add(Dense(6, activation='softmax'))
model_glove_bin.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
# train
history =model_glove_bin.fit(x_train_glove, y_train_binary, validation_data=(x_val_glove, y_val_binary), batch_size = 32, epochs = 12)
# test

y_hats =model_glove_bin.predict(x_test_glove)
actual_y=np.argmax(y_test_six_way,axis=1)
pred_y=np.argmax(y_hats,axis=1)
con_matrix = confusion_matrix(actual_y,pred_y)
print("Confusion Matrix\n",con_matrix)
print('\nAccuracy on glove embedded inputs : {:.4f}\n'.format(accuracy_score(actual_y,pred_y)))
print('\n f1 score on glove embedded inputs : {:.4f}\n'.format(f1_score(actual_y, pred_y,average='macro')))
print('\nClassification Report of glove embedded inputs \n')
print(classification_report(actual_y, pred_y, target_names=['False', 'True', 'barely-true', 'half-true', 'mostly-true', 'pants-fire']))
# report accuracy, f1-score and confusion matrix